In [ ]:
# default_exp dems

In [ ]:
# export
from dataclasses import dataclass
from pathlib import Path

import xarray as xr

from trex_diviner_pipeline import core


class LOLA_DEM:
    root = Path("/luna4/maye/dems")
    fnames = {
        "data": "ldem_128_jp2.tif",
        "slope": "ldem_128_slope_percent.tif",
        "aspect": "ldem_128_aspect.tif",
    }

    def __init__(self, lat_limit=None):
        self.lat_limit = lat_limit

        # assign name attributes data/slope/aspect:
        for key, fname in self.fnames.items():
            setattr(self, f"{key}_fpath", self.root / fname)
            setattr(self, key, core.read_raster_image(self.root/fname).squeeze(drop=True))
            
        lats = np.linspace(-90, 90, len(self.data.y))
        lons = np.linspace(-180, 180, len(self.data.x))
        for key in self.fnames.keys():
            o = getattr(self, key)
            o = o.assign_coords(lat=('y', lats))
            o = o.assign_coords(lon=('x', lons))
            o = o.swap_dims({'y':'lat', 'x':'lon'})
            setattr(self, key, o)
            
    def slice_lat(self, data, lat):
        s = slice(-lat, lat)
        return self.data.sel(lat=s, drop=True)

In [ ]:
dem = LOLA_DEM()

In [ ]:
dem.data

<xarray.DataArray (lat: 23040, lon: 46080)>
dask.array<getitem, shape=(23040, 46080), dtype=int16, chunksize=(2048, 4096), chunktype=numpy.ndarray>
Coordinates:
    y        (lat) float64 2.729e+06 2.729e+06 ... -2.729e+06 -2.729e+06
    x        (lon) float64 -5.458e+06 -5.458e+06 ... 5.458e+06 5.458e+06
  * lat      (lat) float64 -90.0 -89.99 -89.98 -89.98 ... 89.98 89.98 89.99 90.0
  * lon      (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Attributes:
    transform:                  (236.9011752, 0.0, -5458203.076608, 0.0, -236...
    crs:                        +proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 ...
    res:                        (236.9011752, 236.9011752)
    is_tiled:                   1
    nodatavals:                 (-32768.0,)
    scales:                     (0.5,)
    offsets:                    (1737400.0,)
    AREA_OR_POINT:              Area
    DATA_SET_ID:                "LRO-L-LOLA-4-GDR-V1.0"
    INSTRUMENT_ID:              "LOLA"
    INSTRUMENT_NAME:            "LUNAR ORBITER LASER ALTIMETER"
    PRODUCER_INSTITUTION_NAME:  "GODDARD SPACE FLIGHT CENTER"
    PRODUCT_CREATION_TIME:      2017-09-15
    PRODUCT_ID:                 "LDEM_128"
    START_TIME:                 2009-07-13T17:33:17.246
    STOP_TIME:                  2016-11-29T05:48:19
    TARGET_NAME:                MOON

In [ ]:
dem.slice_lat(dem.data, 80)

<xarray.DataArray (lat: 20480, lon: 46080)>
dask.array<getitem, shape=(20480, 46080), dtype=int16, chunksize=(2048, 4096), chunktype=numpy.ndarray>
Coordinates:
    y        (lat) float64 2.426e+06 2.426e+06 ... -2.426e+06 -2.426e+06
    x        (lon) float64 -5.458e+06 -5.458e+06 ... 5.458e+06 5.458e+06
  * lat      (lat) float64 -80.0 -79.99 -79.98 -79.98 ... 79.98 79.98 79.99 80.0
  * lon      (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Attributes:
    transform:                  (236.9011752, 0.0, -5458203.076608, 0.0, -236...
    crs:                        +proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 ...
    res:                        (236.9011752, 236.9011752)
    is_tiled:                   1
    nodatavals:                 (-32768.0,)
    scales:                     (0.5,)
    offsets:                    (1737400.0,)
    AREA_OR_POINT:              Area
    DATA_SET_ID:                "LRO-L-LOLA-4-GDR-V1.0"
    INSTRUMENT_ID:              "LOLA"
    INSTRUMENT_NAME:            "LUNAR ORBITER LASER ALTIMETER"
    PRODUCER_INSTITUTION_NAME:  "GODDARD SPACE FLIGHT CENTER"
    PRODUCT_CREATION_TIME:      2017-09-15
    PRODUCT_ID:                 "LDEM_128"
    START_TIME:                 2009-07-13T17:33:17.246
    STOP_TIME:                  2016-11-29T05:48:19
    TARGET_NAME:                MOON

In [ ]:
# check dtypes
assert isinstance(dem.data_fpath, Path)
assert isinstance(dem.slope_fpath, Path)
assert isinstance(dem.aspect_fpath, Path)
assert isinstance(dem.data, xr.DataArray)

In [ ]:
# check squeezing and resulting dims
assert dem.data.ndim == 2
assert dem.slope.ndim == 2